# Метаклассы

Всё в Python является объектом, и классы не исключение. Это значит, что эти классы кто-то создает.

In [1]:
class ClassA():
    pass

a = ClassA()

Тип объекта `a` является `ClassA`, потому что он создал наш объект:

In [2]:
type(a)

__main__.ClassA

Однако, у класса тоже есть тип, и этот тип `type`, потому что `type` создал наш класс `ClassA`.  
В данном случае, `type` является метаклассом, он создает другие классы.

In [3]:
type(ClassA)

type

А типом `type` является он же сам. Это рекурсивное замыкание которое реализовано с помощью языка C внутри Python:

In [4]:
type(type)

type

Очень важно понимать разницу между созданием и наследованием.  
В данном случае, `ClassA` не является подклассом `type`. `Type` создает `ClassA`, но `ClassA` не наследуется от `type`:

In [5]:
issubclass(ClassA, type)

False

`ClassA` наследуется от класса `object`:

In [6]:
issubclass(ClassA, object)

True

## Создание классов

Чтобы понять, как классы задаются, можно написать простую функцию которая возвращает класс:

In [7]:
def dummy_factory():
    class ClassA():
        pass
    
    return ClassA

Создание класса "на лету":

In [8]:
Dummy = dummy_factory()

Сравнение двух разных объектов. `()` - скобки вызывают у класса метод `__new__` создания нового объекта:

In [9]:
Dummy() is Dummy()

False

Однако, Python работает конечно же не так как выше. Для создания классов используется метакласс `type`.  
Создаем класс `NewClass` без родителей и без каких-либо атрибутов:

In [10]:
NewClass = type('NewClass', (), {})

Поизучаем информацию о только что созданном классе:

In [11]:
print(NewClass)

<class '__main__.NewClass'>


In [12]:
print(NewClass())

In [13]:
print(type(NewClass))

<class 'type'>


## Собственный метакласс и метод `__new__`

Однако, чаще всего классы создаются по другому.  
Собственный метакласс может управлять поведением при создании класса:

In [14]:
class Meta(type): # Наследование от класса type

    # Конструктор класса Meta. Метод определяет момент создания объекта класса Meta.
    # В данном случае, объектом является другой класс (A), поэтому мы можем изменять поведение
    # при создании другого класса. Метод принимает имя класса, его родителей, и его атрибуты:
    def __new__(cls, name, parents, attrs):
        print('Создается класс:', name)

        if 'class_id' not in attrs: # Мы можем при создании класса определить в нем какие либо атрибуты
            attrs['class_id'] = name.lower() # Создаем атрибут для класса

        return super().__new__(cls, name, parents, attrs)


При декларации класса `ClassA` с параметром `metaclass=Meta`, создается КЛАСС (не объект!), о чем выводится сообщение:

In [15]:
class ClassA(metaclass=Meta): # Указываем, что его метаклассом является класс Meta
    pass

Создается класс: ClassA


Также, в классе `ClassA` метаклассом `Meta` создается атрибут:

In [16]:
ClassA.class_id

'classa'

Метаинформация о классе:

In [17]:
print(ClassA)

<class '__main__.ClassA'>


Класс имеет тип `Meta`, потому что класс `ClassA` создал метакласс `Meta`:

In [18]:
print(type(ClassA))

<class '__main__.Meta'>


А экземпляр класса `a` имеет тип `ClassA`, потому что класс `ClassА` создал этот объект:

In [19]:
a = ClassA()
print(type(a))

<class '__main__.ClassA'>


## Собственный метакласс и метод `__init__`

Это метакласс, потому что он наследуется от класса `type`:

In [28]:
class Meta(type):

    # Метод инициализации класса. cls - ссылка на класс Base
    def __init__(cls, name, bases, attrs):
        print('Initializing class - ', name)
        # Мы можем создать при нициализации классов какой-то атрибут класса и что-то в нем, например, логировать
        if not hasattr(cls, 'registry'):
            cls.registry = {}
        else:
            cls.registry[name.lower()] = cls
        super().__init__(name, bases, attrs)

Класс это тоже объект-класс который создается метаклассом:

In [29]:
class Base(metaclass=Meta): pass

Initializing class -  Base


In [30]:
class A(Base): pass

Initializing class -  A


In [31]:
class B(Base): pass

Initializing class -  B


Все эти классы создал класс `Meta`:

In [32]:
type(Base)

__main__.Meta

In [33]:
type(A)

__main__.Meta

In [34]:
type(B)

__main__.Meta

Во время создания классов `Base`, `A`, `B` классом `Meta` был зарегистрирован атрибут класса `registry` (не объекта!):

In [35]:
Base.registry

{'a': __main__.A, 'b': __main__.B}

In [36]:
Base.__subclasses__()

[__main__.A, __main__.B]

Мы можем создать свой класс с помощью метакласса `Meta`:

In [37]:
myA = Meta('MyClassA', (), {})

Initializing class -  MyClassA


In [38]:
myA.registry

{}

In [39]:
myA.__subclasses__()

[]

## Абстрактные методы

Абстрактные методы можно использовать с помощью стандартной библиотеки `abc`:

In [40]:
from abc import ABCMeta, abstractmethod

class Sender(metaclass=ABCMeta):

    @abstractmethod
    def send(self):
        """Do something"""

О чем говорит декоратор `@abstractmethod`? О том, что у нас не получится создать класс не переопределив метод, т.е., мы обязаны реализовать метод в дочерних классах. Если не реализовать метод, то произойдет исключение типа `TypeError`:

In [42]:
class Child(Sender):
    pass

child = Child()

TypeError: Can't instantiate abstract class Child with abstract methods send

Исправим класс `Child` и реализуем в нем метод:

In [43]:
class Child(Sender):
    def send(self):
        pass

child = Child()

Абстрактные методы в Python используются редко, чаще всего вызывается исключение `NotImplementedError`, который говорит о том, что этот метод нужно реализовать в дочерних классах:

In [47]:
class PythonWay():

    def send(self):
        raise NotImplementedError

class Child(PythonWay):
    pass

child = Child()
child.send()

NotImplementedError: 